# Testing with the api

## Setup

In [9]:
import requests
import pprint
import os

### Secrets

In [10]:
AZURE_DEVOPS_TOKEN = "CJRiH2m0qE3DgFxZsWWxt1ZLyZaaJbxEP617wuMXnBKxhuGXYQbbJQQJ99BCACAAAAAjW9BoAAASAZDO43P4"
HEADERS = {
    "Authorization": f"Bearer {AZURE_DEVOPS_TOKEN}"
}

## Models

Prompt
Crea una clase en python con la siguiente informacion para extraerlo de un json


### Project
Project(abbreviation, defaultTeamImageUrl, description, id, lastUpdateTime, name, revision, state, url, visibility)

In [11]:
class Project:
    def __init__(self, abbreviation, defaultTeamImageUrl, description, id, lastUpdateTime, name, revision, state, url, visibility):
        self.abbreviation = abbreviation
        self.defaultTeamImageUrl = defaultTeamImageUrl
        self.description = description
        self.id = id
        self.lastUpdateTime = lastUpdateTime
        self.name = name
        self.revision = revision
        self.state = state
        self.url = url
        self.visibility = visibility

    def __repr__(self):
        return (
            f"Project(abbreviation={self.abbreviation}, defaultTeamImageUrl={self.defaultTeamImageUrl}, "
            f"description={self.description}, id={self.id}, lastUpdateTime={self.lastUpdateTime}, "
            f"name={self.name}, revision={self.revision}, state={self.state}, url={self.url}, "
            f"visibility={self.visibility})"
        )

    @classmethod
    def from_json(cls, json_data):
        return cls(
            abbreviation=json_data.get("abbreviation"),
            defaultTeamImageUrl=json_data.get("defaultTeamImageUrl"),
            description=json_data.get("description"),
            id=json_data.get("id"),
            lastUpdateTime=json_data.get("lastUpdateTime"),
            name=json_data.get("name"),
            revision=json_data.get("revision"),
            state=json_data.get("state"),
            url=json_data.get("url"),
            visibility=json_data.get("visibility"),
        )
    

### Team
WebApiTeam(description, id, identity, identityUrl, name, projectId, projectName, url)

In [12]:
class WebApiTeam:
    '''
    ALL TEAMS 
    GET https://dev.azure.com/{organization}/_apis/teams?api-version=7.2-preview.3
    MY TEAMS
    GET https://dev.azure.com/{organization}/_apis/teams?api-version=7.2-preview.3

    '''
    def __init__(self, description, id, identity, identityUrl, name, projectId, projectName, url):
        self.description = description
        self.id = id
        self.identity = identity
        self.identityUrl = identityUrl
        self.name = name
        self.projectId = projectId
        self.projectName = projectName
        self.url = url

    def __repr__(self):
        return (
            f"WebApiTeam(description={self.description}, id={self.id}, identity={self.identity}, "
            f"identityUrl={self.identityUrl}, name={self.name}, projectId={self.projectId}, "
            f"projectName={self.projectName}, url={self.url})"
        )

    @classmethod
    def from_json(cls, json_data):
        return cls(
            description=json_data.get("description"),
            id=json_data.get("id"),
            identity=json_data.get("identity"),
            identityUrl=json_data.get("identityUrl"),
            name=json_data.get("name"),
            projectId=json_data.get("projectId"),
            projectName=json_data.get("projectName"),
            url=json_data.get("url"),
        )

### TeamMember

In [13]:
class IdentityRef:
    def __init__(self, _links, descriptor, directoryAlias, displayName, id, imageUrl, inactive, isAadIdentity, isContainer, isDeletedInOrigin, profileUrl, uniqueName, url):
        self._links = _links
        self.descriptor = descriptor
        self.directoryAlias = directoryAlias
        self.displayName = displayName
        self.id = id
        self.imageUrl = imageUrl
        self.inactive = inactive
        self.isAadIdentity = isAadIdentity
        self.isContainer = isContainer
        self.isDeletedInOrigin = isDeletedInOrigin
        self.profileUrl = profileUrl
        self.uniqueName = uniqueName
        self.url = url

    def __repr__(self):
        return (
            #f"IdentityRef(_links={self._links}, descriptor={self.descriptor}, directoryAlias={self.directoryAlias}, "
            f"displayName={self.displayName}, id={self.id}, imageUrl={self.imageUrl}, inactive={self.inactive}, "
            #f"isAadIdentity={self.isAadIdentity}, isContainer={self.isContainer}, isDeletedInOrigin={self.isDeletedInOrigin}, "
            f"profileUrl={self.profileUrl}, uniqueName={self.uniqueName}, url={self.url})"
        )

    @classmethod
    def from_json(cls, json_data):
        return cls(
            _links=json_data.get("_links"),
            descriptor=json_data.get("descriptor"),
            directoryAlias=json_data.get("directoryAlias"),
            displayName=json_data.get("displayName"),
            id=json_data.get("id"),
            imageUrl=json_data.get("imageUrl"),
            inactive=json_data.get("inactive"),
            isAadIdentity=json_data.get("isAadIdentity"),
            isContainer=json_data.get("isContainer"),
            isDeletedInOrigin=json_data.get("isDeletedInOrigin"),
            profileUrl=json_data.get("profileUrl"),
            uniqueName=json_data.get("uniqueName"),
            url=json_data.get("url"),
        )
    
    def to_dict(self):
        return {
            "id": self.id,
            "displayName": self.displayName,
            "imageUrl": self.imageUrl,
        }


In [14]:
class TeamMember:
    def __init__(self, identity, isTeamAdmin):
        self.identity = identity
        self.isTeamAdmin = isTeamAdmin

    def __repr__(self):
        return f"TeamMember(identity={self.identity}, isTeamAdmin={self.isTeamAdmin})"

    @classmethod
    def from_json(cls, json_data):
        return cls(
            identity=IdentityRef.from_json(json_data.get("identity")),
            isTeamAdmin=json_data.get("isTeamAdmin"),
        )
    
    def to_dict(self):
        return {
            "isTeamAdmin": self.isTeamAdmin == True,
            "id": self.identity.id,
            "displayName": self.identity.displayName,
            "imageUrl": self.identity.imageUrl,
        }


### WorkItem

In [61]:
class WorkItem:
    def __init__(self, _links, commentVersionRef, fields, id, relations, rev, url, organization=None, project=None):
        self._links = _links
        self.commentVersionRef = commentVersionRef
        self.fields = fields
        self.id = id
        self.relations = relations
        self.rev = rev
        self.url = url

        self.organization = organization
        self.project = project

        self.completedHours = None
        self.title = None
        self.state = None

    def getDetails(self,headers):
        #Makes a request to get the details we need
        response = requests.get(self.url, headers=headers)#Cuidado con el header
        fields = response.json().get("fields")
        
        self.completedHours = fields["Microsoft.VSTS.Scheduling.CompletedWork"]
        self.title = fields["System.Title"]
        self.state = fields["System.State"]

    def __repr__(self):
        return (
            f"WorkItem(id={self.id}, url={self.url}, title={self.title}, state={self.state}, completedHours={self.completedHours})"
        )

    @classmethod
    def from_json(cls, json_data):
        return cls(
            _links=json_data.get("_links"),
            commentVersionRef=json_data.get("commentVersionRef"),
            fields=json_data.get("fields"),
            id=json_data.get("id"),
            relations=json_data.get("relations"),
            rev=json_data.get("rev"),
            url=json_data.get("url"),
        )

In [ ]:
# Parse the tasks into an array of WorkItem objects
work_items = []
if tasks and "workItems" in tasks:
    for item in tasks["workItems"]:
        work_item = WorkItem.from_json(item)
        work_items.append(work_item)

# Display the parsed work items
work_items

## Test

### Send request
- GET https://dev.azure.com/{organization}/{project}/_apis/wit/workitemtypecategories?api-version=7.2-preview.2
- GET https://dev.azure.com/{organization}/_apis/wit/workitemtypes/{type}/states?api-version=7.2-preview.1

#### Backend

In [28]:
def make_request(method, url, **kwargs):
    try:
        if method.upper() == "GET":
            response = requests.get(url, headers=HEADERS, **kwargs)
            print(response)
        elif method.upper() == "POST":
            response = requests.post(url, headers=HEADERS, **kwargs)
        else:
            print(f"Unsupported HTTP method: {method}")
            return None
        
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        return response.json()  # Parse the response as JSON
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


def makeRequest():

    cmd = input("Enter the command (e.g., GET https://example.com): ").strip()
    cmd = cmd.replace('%7B', '{')
    cmd = cmd.replace('%7D', '}')
    cmd = cmd.replace('{{', '{')
    cmd = cmd.replace('}}', '}')
    clear = lambda: os.system('cls')
    clear()

    print(cmd)
    

    cmd = cmd.split()
    method = cmd[0]
    url = cmd[1] 
    
    #Remplazar valores
    variables = {
        "organization": "FSO-DnA-Devops",
        "projectId": "e4005fd0-7b95-4391-8486-c4b21c935b2e",
        "project": "e4005fd0-7b95-4391-8486-c4b21c935b2e",
        "api-version": "7.3",
        "baseUrl": "dev.azure.com",
    }

    i=0
    while i<len(url):
        if url[i] == '{':
            key = ''
            i+=1
            while url[i] != '}':
                key += url[i]
                i+=1
            if key not in variables:
                value = input(f"Enter value for {key}: ").strip()
                variables[key] = value
        i+=1

    formatted_url = url.format(**variables)
    print(formatted_url)
    
    result = make_request(method, formatted_url)
    if result:
        pprint.pprint(result)

    return result

In [44]:
response_teams = None
try:
    response_teams = makeRequest()
except:
    print("An error occurred")
#GET https://dev.azure.com/{organization}/_apis/teams?api-version=7.2-preview.3

GET https://dev.azure.com/{organization}/_apis/teams?api-version=7.2-preview.3
https://dev.azure.com/FSO-DnA-Devops/_apis/teams?api-version=7.2-preview.3
<Response [200]>
{'count': 23,
 'value': [{'description': 'The default project team.',
            'id': 'ef72e621-835e-40c1-90f8-39eae8e9244c',
            'identityUrl': 'https://spsprodsbr1.vssps.visualstudio.com/A312f46bb-6f01-4060-9a62-4a8f3f52893d/_apis/Identities/ef72e621-835e-40c1-90f8-39eae8e9244c',
            'name': 'Pilar Engenharia de Dados Team',
            'projectId': '3060f42c-be4b-482e-ac9d-21f05e449d71',
            'projectName': 'Pilar Engenharia de Dados',
            'url': 'https://dev.azure.com/FSO-DnA-Devops/_apis/projects/3060f42c-be4b-482e-ac9d-21f05e449d71/teams/ef72e621-835e-40c1-90f8-39eae8e9244c'},
           {'description': '',
            'id': '2331f228-f2fd-4415-be34-137b3d08eccc',
            'identityUrl': 'https://spsprodsbr1.vssps.visualstudio.com/A312f46bb-6f01-4060-9a62-4a8f3f52893d/_apis/Id

In [30]:
teams = []

def find_project_teams(result,project_id):
    if not result or 'value' not in result:
        print("No valid result data to search.")
        return
       
    for team in result["value"]:
        if team.get('projectId') == project_id:
            teams.append(WebApiTeam.from_json(team))


find_project_teams(response_teams,"e4005fd0-7b95-4391-8486-c4b21c935b2e")

In [31]:
teams

[WebApiTeam(description=, id=2331f228-f2fd-4415-be34-137b3d08eccc, identity=None, identityUrl=https://spsprodsbr1.vssps.visualstudio.com/A312f46bb-6f01-4060-9a62-4a8f3f52893d/_apis/Identities/2331f228-f2fd-4415-be34-137b3d08eccc, name=Proposta, projectId=e4005fd0-7b95-4391-8486-c4b21c935b2e, projectName=Generative AI, url=https://dev.azure.com/FSO-DnA-Devops/_apis/projects/e4005fd0-7b95-4391-8486-c4b21c935b2e/teams/2331f228-f2fd-4415-be34-137b3d08eccc),
 WebApiTeam(description=, id=f11bf745-d002-4535-8b02-a6456be0901e, identity=None, identityUrl=https://spsprodsbr1.vssps.visualstudio.com/A312f46bb-6f01-4060-9a62-4a8f3f52893d/_apis/Identities/f11bf745-d002-4535-8b02-a6456be0901e, name=Interno, projectId=e4005fd0-7b95-4391-8486-c4b21c935b2e, projectName=Generative AI, url=https://dev.azure.com/FSO-DnA-Devops/_apis/projects/e4005fd0-7b95-4391-8486-c4b21c935b2e/teams/f11bf745-d002-4535-8b02-a6456be0901e),
 WebApiTeam(description=Desenvolver uma solução voltada a retenção de clientes para a

In [32]:
#Get teams member
'''GET https://dev.azure.com/{organization}/_apis/projects/{projectId}/teams/{teamId}/members?api-version=7.2-preview.2
GET https://dev.azure.com/{organization}/_apis/projects/{projectId}/teams/9083e8b0-af44-4f90-9bdd-f54f9bb431f2/members?api-version=7.2-preview.2
'''
members = None
try:
    members = makeRequest()
except:
    print("An error occurred")

GET https://dev.azure.com/{organization}/_apis/projects/{projectId}/teams/9083e8b0-af44-4f90-9bdd-f54f9bb431f2/members?api-version=7.2-preview.2
https://dev.azure.com/FSO-DnA-Devops/_apis/projects/e4005fd0-7b95-4391-8486-c4b21c935b2e/teams/9083e8b0-af44-4f90-9bdd-f54f9bb431f2/members?api-version=7.2-preview.2
<Response [200]>
{'count': 35,
 'value': [{'identity': {'_links': {'avatar': {'href': 'https://dev.azure.com/FSO-DnA-Devops/_apis/GraphProfile/MemberAvatars/aad.YTQ3YzdjMmMtMWRkNy03NThkLWFlZmYtMDQzOWQ0NzY0MWEx'}},
                         'descriptor': 'aad.YTQ3YzdjMmMtMWRkNy03NThkLWFlZmYtMDQzOWQ0NzY0MWEx',
                         'displayName': 'Thamires Azeredo',
                         'id': 'a47c7c2c-1dd7-658d-aeff-0439d47641a1',
                         'imageUrl': 'https://dev.azure.com/FSO-DnA-Devops/_api/_common/identityImage?id=a47c7c2c-1dd7-658d-aeff-0439d47641a1',
                         'uniqueName': 'Thamires.Azeredo@br.ey.com',
                         'url': 'htt

In [33]:
def parse_members(result):
    for m in result["value"]:
        member = TeamMember.from_json(m)
        print(member.to_dict())


parse_members(members)

{'isTeamAdmin': True, 'id': 'a47c7c2c-1dd7-658d-aeff-0439d47641a1', 'displayName': 'Thamires Azeredo', 'imageUrl': 'https://dev.azure.com/FSO-DnA-Devops/_api/_common/identityImage?id=a47c7c2c-1dd7-658d-aeff-0439d47641a1'}
{'isTeamAdmin': False, 'id': 'c490a36e-382a-69ac-8012-08a3f8538d29', 'displayName': 'Ibrahim Dib', 'imageUrl': 'https://dev.azure.com/FSO-DnA-Devops/_api/_common/identityImage?id=c490a36e-382a-69ac-8012-08a3f8538d29'}
{'isTeamAdmin': False, 'id': '8130a876-d6e4-659e-8ea2-0beed6d924ac', 'displayName': 'Maria Caceres', 'imageUrl': 'https://dev.azure.com/FSO-DnA-Devops/_api/_common/identityImage?id=8130a876-d6e4-659e-8ea2-0beed6d924ac'}
{'isTeamAdmin': False, 'id': 'c42eb0f2-e93e-6c1e-86de-0d3b68c8f4ad', 'displayName': 'Levi Araujo', 'imageUrl': 'https://dev.azure.com/FSO-DnA-Devops/_api/_common/identityImage?id=c42eb0f2-e93e-6c1e-86de-0d3b68c8f4ad'}
{'isTeamAdmin': False, 'id': '1f2fca06-18ba-6192-b34a-12d0682cbda6', 'displayName': 'Luccas Claudio', 'imageUrl': 'https:/

In [ ]:
#tendria que seleccionar un equipo o sino ver de todos

# END

In [ ]:
variables = {
        "organization": "FSO-DnA-Devops",
        "projectId": "e4005fd0-7b95-4391-8486-c4b21c935b2e",
        "project": "e4005fd0-7b95-4391-8486-c4b21c935b2e",
        "api-version": "7.3",
        "baseUrl": "dev.azure.com",
    }

In [ ]:
queries = {
    "getWIPwithSpecificStatus": """SELECT
    [System.Id],
    [System.WorkItemType],
    [System.Title],
    [System.AssignedTo],
    [System.State],
    [System.Tags]
FROM workitems
WHERE
    [System.TeamProject] = 'Generative AI'
    AND [System.ChangedDate] > @today - 180
    AND [System.WorkItemType] = 'Task'
    AND [System.State] = '{Active}'
ORDER BY [System.Id] asc
""",
    "getWIPFromMember": """SELECT
    [System.Id],
    [System.WorkItemType],
    [System.Title],
    [System.AssignedTo],
    [System.State],
    [System.Tags]
FROM workitems
WHERE
    [System.TeamProject] = 'Generative AI'
    AND [System.ChangedDate] > @today - 180
    AND [System.WorkItemType] = 'Task'
    AND [System.AssignedTo] = '{Member}'
ORDER BY [System.Id] asc
"""
}

### Obtener las tareas del usuario

In [65]:
def get_tasks(organization, project, team, userName):
    url = f"https://dev.azure.com/{organization}/{project}/{team}/_apis/wit/wiql?api-version=5.1&fields=System.State"
    query = {
        "query": f"""SELECT
    [System.Id],
    [System.WorkItemType],
    [System.Title],
    [System.AssignedTo],
    [System.State],
    [System.Tags],
    [Microsoft.VSTS.Scheduling.CompletedWork]
FROM workitems
WHERE
    [System.TeamProject] = 'Generative AI'
    AND [System.ChangedDate] > @today - 180
    AND [System.WorkItemType] = 'Task'
    AND [System.AssignedTo] = '{userName}'
ORDER BY [System.Id] asc
"""
    }

    #    AND [System.ChangedDate] > @today - 180
    
    try:
        response = requests.post(url, headers=HEADERS, json=query)
        response.raise_for_status()
        tasks = response.json()
        #pprint.pprint(tasks)
        return tasks
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
organization = "FSO-DnA-Devops"
project = "e4005fd0-7b95-4391-8486-c4b21c935b2e"
team = "9083e8b0-af44-4f90-9bdd-f54f9bb431f2"
userName = "Thamires Azeredo"
tasks = get_tasks(organization, project, team,userName)

# Parse the tasks into an array of WorkItem objects
work_items = []
if tasks and "workItems" in tasks:
    for item in tasks["workItems"]:
        work_item = WorkItem.from_json(item)
        work_item.getDetails(HEADERS)
        work_items.append(work_item)
        print(work_item)
        break

# Display the parsed work items



WorkItem(id=1079, url=https://dev.azure.com/FSO-DnA-Devops/e4005fd0-7b95-4391-8486-c4b21c935b2e/_apis/wit/workItems/1079, title=Preparação e solicitação de acessos, state=Closed, completedHours=8.0)


8


In [64]:
url = work_items[0].url
response = requests.get(url, headers=HEADERS)

if response.status_code == 200:
    data = response.json()
    pprint.pprint(data)
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

{'_links': {'fields': {'href': 'https://dev.azure.com/FSO-DnA-Devops/e4005fd0-7b95-4391-8486-c4b21c935b2e/_apis/wit/fields'},
            'html': {'href': 'https://dev.azure.com/FSO-DnA-Devops/e4005fd0-7b95-4391-8486-c4b21c935b2e/_workitems/edit/1079'},
            'self': {'href': 'https://dev.azure.com/FSO-DnA-Devops/e4005fd0-7b95-4391-8486-c4b21c935b2e/_apis/wit/workItems/1079'},
            'workItemComments': {'href': 'https://dev.azure.com/FSO-DnA-Devops/e4005fd0-7b95-4391-8486-c4b21c935b2e/_apis/wit/workItems/1079/comments'},
            'workItemRevisions': {'href': 'https://dev.azure.com/FSO-DnA-Devops/e4005fd0-7b95-4391-8486-c4b21c935b2e/_apis/wit/workItems/1079/revisions'},
            'workItemType': {'href': 'https://dev.azure.com/FSO-DnA-Devops/e4005fd0-7b95-4391-8486-c4b21c935b2e/_apis/wit/workItemTypes/Task'},
            'workItemUpdates': {'href': 'https://dev.azure.com/FSO-DnA-Devops/e4005fd0-7b95-4391-8486-c4b21c935b2e/_apis/wit/workItems/1079/updates'}},
 'fields':